# PostgreSQL Analytics with Lakehouse Lab

This notebook demonstrates PostgreSQL integration for analytics workloads, combining SQL with Python data science tools.

## What's Covered

- **PostgreSQL**: Relational database integration
- **DuckDB**: High-performance analytics on PostgreSQL data
- **Cross-system queries**: PostgreSQL + S3 data federation
- **Data visualization**: Charts and dashboards


In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import duckdb
import psycopg2
from sqlalchemy import create_engine
import os

# Configure plotting
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
%matplotlib inline

print("✅ PostgreSQL Analytics Environment Ready!")
print(f"📊 DuckDB version: {duckdb.__version__}")
print(f"🐘 psycopg2 version: {psycopg2.__version__}")

## Connect to PostgreSQL

In [ ]:
# PostgreSQL connection with environment variables
pg_host = 'postgres'
pg_user = os.environ.get('POSTGRES_USER', 'postgres')
pg_password = os.environ.get('POSTGRES_PASSWORD', 'postgres')
pg_database = os.environ.get('POSTGRES_DB', 'lakehouse')
pg_port = 5432

# Create SQLAlchemy engine
pg_engine = create_engine(
    f'postgresql://{pg_user}:{pg_password}@{pg_host}:{pg_port}/{pg_database}'
)

# Test connection
try:
    with pg_engine.connect() as conn:
        result = conn.execute("SELECT version()")
        version = result.fetchone()[0]
        print(f"✅ Connected to PostgreSQL: {version.split(',')[0]}")
except Exception as e:
    print(f"❌ PostgreSQL connection failed: {e}")
    print("Make sure PostgreSQL service is running and accessible")

## Create Sample PostgreSQL Data

In [ ]:
# Create sample tables in PostgreSQL
try:
    with pg_engine.connect() as conn:
        with conn.begin():
            # Create customers table
            conn.execute("""
                DROP TABLE IF EXISTS orders CASCADE;
                DROP TABLE IF EXISTS customers CASCADE;
                
                CREATE TABLE customers (
                    customer_id SERIAL PRIMARY KEY,
                    customer_name VARCHAR(100) NOT NULL,
                    email VARCHAR(100) UNIQUE,
                    city VARCHAR(50),
                    country VARCHAR(50),
                    signup_date DATE DEFAULT CURRENT_DATE
                );
            """)
            
            # Create orders table
            conn.execute("""
                CREATE TABLE orders (
                    order_id SERIAL PRIMARY KEY,
                    customer_id INTEGER REFERENCES customers(customer_id),
                    order_date DATE DEFAULT CURRENT_DATE,
                    product_name VARCHAR(100),
                    quantity INTEGER,
                    unit_price DECIMAL(10,2),
                    total_amount DECIMAL(10,2)
                );
            """)
            
            # Insert sample customers
            conn.execute("""
                INSERT INTO customers (customer_name, email, city, country) VALUES
                ('Alice Johnson', 'alice@example.com', 'New York', 'USA'),
                ('Bob Smith', 'bob@example.com', 'London', 'UK'),
                ('Carol Brown', 'carol@example.com', 'Toronto', 'Canada'),
                ('David Wilson', 'david@example.com', 'Sydney', 'Australia'),
                ('Eva Martinez', 'eva@example.com', 'Madrid', 'Spain');
            """)
            
            # Insert sample orders
            conn.execute("""
                INSERT INTO orders (customer_id, product_name, quantity, unit_price, total_amount) VALUES
                (1, 'Laptop', 1, 999.99, 999.99),
                (1, 'Mouse', 2, 25.50, 51.00),
                (2, 'Keyboard', 1, 79.99, 79.99),
                (2, 'Monitor', 1, 299.99, 299.99),
                (3, 'Headphones', 1, 149.99, 149.99),
                (4, 'Tablet', 1, 399.99, 399.99),
                (5, 'Phone', 1, 699.99, 699.99);
            """)
            
    print("✅ Sample PostgreSQL data created successfully")
except Exception as e:
    print(f"❌ Failed to create sample data: {e}")
    print(f"Error details: {str(e)}")

## PostgreSQL Analytics Queries

In [ ]:
# Query PostgreSQL data with pandas
customers_df = pd.read_sql("""
    SELECT * FROM customers ORDER BY customer_id
""", pg_engine)

print("Customers in PostgreSQL:")
display(customers_df)

orders_df = pd.read_sql("""
    SELECT * FROM orders ORDER BY order_id
""", pg_engine)

print("\nOrders in PostgreSQL:")
display(orders_df)

In [ ]:
# Advanced analytics query
customer_analytics = pd.read_sql("""
    SELECT 
        c.country,
        COUNT(DISTINCT c.customer_id) as customer_count,
        COUNT(o.order_id) as total_orders,
        SUM(o.total_amount) as total_revenue,
        AVG(o.total_amount) as avg_order_value
    FROM customers c
    LEFT JOIN orders o ON c.customer_id = o.customer_id
    GROUP BY c.country
    ORDER BY total_revenue DESC
""", pg_engine)

print("Customer Analytics by Country:")
display(customer_analytics)

# Visualize the data
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Revenue by country
ax1.bar(customer_analytics['country'], customer_analytics['total_revenue'])
ax1.set_title('Total Revenue by Country')
ax1.set_xlabel('Country')
ax1.set_ylabel('Revenue ($)')
ax1.tick_params(axis='x', rotation=45)

# Average order value by country
ax2.bar(customer_analytics['country'], customer_analytics['avg_order_value'])
ax2.set_title('Average Order Value by Country')
ax2.set_xlabel('Country')
ax2.set_ylabel('Avg Order Value ($)')
ax2.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

## DuckDB + PostgreSQL Integration

In [ ]:
# Connect to DuckDB and configure S3
duck_conn = duckdb.connect()

# Configure S3 access with environment variables
minio_user = os.environ.get('MINIO_ROOT_USER', 'minio')
minio_password = os.environ.get('MINIO_ROOT_PASSWORD', 'minio123')

duck_conn.execute("""
    INSTALL httpfs;
    LOAD httpfs;
    SET s3_endpoint='minio:9000';
    SET s3_use_ssl=false;
    SET s3_url_style='path';
""")

duck_conn.execute(f"SET s3_access_key_id='{minio_user}';")
duck_conn.execute(f"SET s3_secret_access_key='{minio_password}';")

print("✅ DuckDB configured for S3 access")

# Install and configure PostgreSQL extension
try:
    duck_conn.execute("INSTALL postgres;")
    duck_conn.execute("LOAD postgres;")
    print("✅ PostgreSQL extension loaded in DuckDB")
except Exception as e:
    print(f"PostgreSQL extension not available: {e}")
    print("Will demonstrate with pandas DataFrame instead")

In [ ]:
# Demonstrate cross-system analytics
# Load PostgreSQL data into DuckDB for analysis
duck_conn.register('pg_customers', customers_df)
duck_conn.register('pg_orders', orders_df)

# Query S3 data and join with PostgreSQL data
cross_system_analysis = duck_conn.execute("""
    WITH s3_orders AS (
        SELECT 
            customer_name,
            product_category,
            total_amount as s3_amount
        FROM read_csv_auto('s3://lakehouse/raw-data/sample_orders.csv')
    ),
    pg_summary AS (
        SELECT 
            c.customer_name,
            c.country,
            SUM(o.total_amount) as pg_amount
        FROM pg_customers c
        LEFT JOIN pg_orders o ON c.customer_id = o.customer_id
        GROUP BY c.customer_name, c.country
    )
    SELECT 
        pg.customer_name,
        pg.country,
        pg.pg_amount,
        COUNT(s3.s3_amount) as s3_order_count,
        SUM(s3.s3_amount) as s3_total
    FROM pg_summary pg
    LEFT JOIN s3_orders s3 ON pg.customer_name = s3.customer_name
    GROUP BY pg.customer_name, pg.country, pg.pg_amount
    ORDER BY pg.pg_amount DESC
""").fetchdf()

print("Cross-System Analytics (PostgreSQL + S3):")
display(cross_system_analysis)

## Advanced PostgreSQL Analytics

In [ ]:
# Time-based analysis
time_analysis = pd.read_sql("""
    SELECT 
        DATE_TRUNC('day', order_date) as order_day,
        COUNT(*) as daily_orders,
        SUM(total_amount) as daily_revenue,
        AVG(total_amount) as avg_order_value
    FROM orders
    GROUP BY DATE_TRUNC('day', order_date)
    ORDER BY order_day
""", pg_engine)

print("Daily Analytics:")
display(time_analysis)

# Product performance
product_analysis = pd.read_sql("""
    SELECT 
        product_name,
        COUNT(*) as order_count,
        SUM(quantity) as total_quantity,
        SUM(total_amount) as total_revenue,
        AVG(unit_price) as avg_price
    FROM orders
    GROUP BY product_name
    ORDER BY total_revenue DESC
""", pg_engine)

print("\nProduct Performance:")
display(product_analysis)

# Visualize product performance
plt.figure(figsize=(12, 8))
plt.subplot(2, 2, 1)
plt.bar(product_analysis['product_name'], product_analysis['total_revenue'])
plt.title('Revenue by Product')
plt.xticks(rotation=45)
plt.ylabel('Revenue ($)')

plt.subplot(2, 2, 2)
plt.bar(product_analysis['product_name'], product_analysis['total_quantity'])
plt.title('Quantity Sold by Product')
plt.xticks(rotation=45)
plt.ylabel('Quantity')

plt.subplot(2, 2, 3)
plt.bar(product_analysis['product_name'], product_analysis['avg_price'])
plt.title('Average Price by Product')
plt.xticks(rotation=45)
plt.ylabel('Price ($)')

plt.subplot(2, 2, 4)
sizes = product_analysis['total_revenue']
plt.pie(sizes, labels=product_analysis['product_name'], autopct='%1.1f%%')
plt.title('Revenue Distribution')

plt.tight_layout()
plt.show()

## Next Steps

1. **Explore more PostgreSQL features**: Window functions, CTEs, advanced joins
2. **Scale with DuckDB**: Use DuckDB for heavy analytics on PostgreSQL exports
3. **Real-time dashboards**: Connect Superset to PostgreSQL for live dashboards
4. **Data pipelines**: Use Airflow to orchestrate PostgreSQL → S3 → DuckDB workflows

## Key Features Demonstrated

✅ **PostgreSQL Integration**: Direct connection with environment variables  
✅ **Cross-system Analytics**: PostgreSQL + S3 data federation with DuckDB  
✅ **Advanced Visualizations**: Multi-panel charts and analysis  
✅ **Real Analytics**: Customer segmentation, product performance, time series  

Happy PostgreSQL analytics!